# web scraping


In [ ]:
import requests
from bs4 import BeautifulSoup
workbook=xlsxwriter.Workbook('ThePrint.xlsx')
worksheet=workbook.add_worksheet()
row=0
column=0
worksheet.write(row,column,"Heading")
worksheet.write(row,column+1,"Body")
worksheet.write(row,column+2,"Category")
worksheet.write(row,column+3,"URL")
row+=1
    
r=requests.get('https://theprint.in', headers=HEADERS)
    urls_to_visit=[]
    unique_urls={}
    count=0
    try:
        if(r.status_code==200):
            soup=BeautifulSoup(r.text, 'html.parser')
            for url in soup.findAll('a'):
                try:
                    if(url.has_attr('href')):
                        if("video" not in url['href'].split("/") and "tag" not in url['href'].split("/") and "author" not in url['href'].split("/")):
                            if(url['href'][0]=='/' and "https://theprint.in"+url['href'] not in unique_urls.keys()):
                                unique_urls["https://theprint.in"+url['href']]=True
                                urls_to_visit.append("https://theprint.in"+url['href'])
                            elif(url['href'][0]=='h' and url['href'].split("/")[2]=="theprint.in" and url['href'] not in unique_urls.keys()):
                                unique_urls[url['href']]=True
                                urls_to_visit.append(url['href'])

# OCR

In [ ]:
from pytesseract import pytesseract
import os
class OCR:
    def _init_(self):
        self.path=r"C:/Program Files/Tesseract-OCR/tesseract.exe"
    def extract(self, filename):
        try:
            pytesseract.tesseract_cmd= self.path
            text=pytesseract.image_to_string(filename)
            return text
        except Exception as e:
            print(e)
            return "Error"
ocr=OCR()
text=ocr.extract("temp_img.png")
print(text)

# MODEL

In [1]:
import nltk
from langdetect import detect
from rake_nltk import Rake
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from twilio.rest import Client
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime  
import pandas as pd
import numpy as np


news_article = """
The healthcare system is overwhelmed due to the surge in COVID-19 cases.
Hospitals are running out of beds and ventilators, putting patients' lives at risk.
Medical staff are exhausted and facing burnout from the prolonged crisis."""
news_article_2 = """
School closures continue due to the ongoing pandemic, causing disruptions in students' education.
Many students are struggling with remote learning and lack access to necessary resources.
Teachers are facing challenges in delivering quality education in virtual classrooms."""
news_article_3 =  """
The Indian Finance Department reported a significant decrease in tax revenues for the current fiscal year.
The national budget deficit has reached an alarming level, raising concerns about fiscal stability.
The Indian rupee has depreciated sharply against major foreign currencies, impacting international trade."""
news_article_4 ="""
The Indian Transportation Department is grappling with a severe shortage of funds for infrastructure maintenance.
Major highways across the country are in a state of disrepair, causing frequent accidents and traffic jams.
Public transport services have been unreliable, leading to commuter frustration and inconvenience."""
news_article_5 = """
A new restaurant opened in the city, promising a unique dining experience.
Food critics have praised the chef's culinary skills."""
nltk.download('punkt')
# Initialize counters for negative and positive statements
negative_statements_count = 0
positive_statements_count = 0

# Language Detection
def detect_language(text):
    try:
        language = detect(text)
        return language
    except:
        return None

# Keyword Extraction
def extract_keywords(text):
    r = Rake()
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()

# Sentiment Analysis for Negative News Detection
def count_sentiments(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(text)
    compound_score = sentiment_scores['compound']
    
    if compound_score <= -0.05:  # Set a threshold for negative sentiment
        return 'Negative'
    elif compound_score >= 0.05:  # Set a threshold for positive sentiment
        return 'Positive'
    else:
        return 'Neutral'

# Categorize News Stories
def categorize_department(text):
    # Define predefined tags for departments
    predefined_tags = {
        'healthcare': 'Health Department',
        "spread":'Health Department',
        "fever":'Health Department',
        "hospital":'Health Department',
        "disease":'Health Department',
        "covid 19":'Health Department',
        'virus':'Health Department',
        'education': 'Education Department',
        'school':'Education Department',
        'college': 'Education Department',
        'study': 'Education Department',
        'course': 'Education Department',
        
        'economy': 'Finance Department',
        'rupee': 'Finance Department',
        'Indian Finance Department': 'Finance Department',
        'national budget': 'Finance Department',
        'investors':'Finance Department',
        
        'finnance': 'Finance Department',
        'goverment loss': 'Finance Department',
        'govt loss': 'Finance Department',
        
        'stockmarket': 'Finance Department',
        'transport':'Transportation Department',
        'road':'Transportation Department',
        "bus":'Transportation Department',
        "car":'Transportation Department',
        'Indian Transportation Department':'Transportation Department',
        'frequent accidents':'Transportation Department',
        "bike":'Transportation Department',
        "train":'Transportation Department',
        "truck":'Transportation Department',
        "trafic light":'Transportation Department',
        "auto":'Transportation Department',
        
    }
    # Check for keywords in the text and assign departments
    for keyword, department in predefined_tags.items():
        if keyword in text.lower():
            return department
    return None

# Department Contacts Mapping
department_contacts = {
    'Health Department': {
        'email': 'nandhakrishnannk63@gmail.com',
        'phone': '+919976160694',
    },
    'Education Department': {
        'email': 'nandhakrishnannk63@gmail.com',
        'phone': '+919976160694',
    },
    'Finance Department': {
        'email': 'nandhakrishnannk63@gmail.com',
        'phone': '+919976160694',
    },
    'Transportation Department': {
        'email': 'nandhakrishnannk63@gmail.com',
        'phone': '+919976160694',
    },
     'Agriculture Department': {
        'email': 'nandhakrishnannk63@gmail.com',
        'phone': '+919976160694',
    },
}

# Function to send SMS notification with a message header
def send_sms_notification(message, department):
    if department in department_contacts:
        # Get the phone number for the department
        recipient_phone_number = department_contacts[department]['phone']

        # Twilio credentials (replace with your own)
        twilio_account_sid = 'AC7965e5ee2c7b95a42fe131c2a92f6604'
        twilio_auth_token = 'b581ad88190bb5a68fc3ca5b35bbaa2c'
        twilio_phone_number = '+12569065210'
        recipient_phone_number = '+919976160694'

        # Add a message header based on the department
        if department == 'Finance Department':
            message = f"[Finance News] {message}"
       
      
        
        current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        message = f"{current_date}\n{message}"

        # Initialize the Twilio client
        client = Client(twilio_account_sid, twilio_auth_token)

        try:
            # Send the SMS notification
            message = client.messages.create(
                to=recipient_phone_number,
                from_=twilio_phone_number,
                body=message
            )

            print(f"SMS Notification sent to {department}. Message SID: {message.sid}")

        except Exception as e:
            print(f"Error sending SMS notification to {department}: {str(e)}")

# Function to send email notification with a message header
def send_email_notification(message, department):
    if department in department_contacts:
        # Get the email address for the department
        recipient_email = department_contacts[department]['email']

        # Email credentials (replace with your own)
        email_address = 'nandhakrishnan.n.2021.ad@ritchennai.edu.in'
        email_password = 'Welcome@123'

        # Compose the email message
        subject = f"Negative News Detected - {department}"

        # Add a message header based on the department
        if department == 'Finance Department':
            subject = f"[Finance News] {subject}"
        if department == 'Health Department':
            subject = f"[Health News] {subject}"
        if department == 'Education Department':
            subject = f"[Education News] {subject}"
        if department == 'Transportation Department':
            subject = f"[Transportation] {subject}"
        if department == 'Agriculture Department':
            subject = f"[Agriculture News] {subject}"
            
        current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        message_body = f"{current_date}\n{message}"
        message_body = message

        # Create the email message
        msg = MIMEMultipart()
        msg['From'] = email_address
        msg['To'] = recipient_email
        msg['Subject'] = subject

        # Attach the message body
        msg.attach(MIMEText(message_body, 'plain'))

        try:
            # Initialize the SMTP server and send the email
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(email_address, email_password)
            text = msg.as_string()
            server.sendmail(email_address, recipient_email, text)
            server.quit()

            print(f"Email notification sent to {department}.")

        except Exception as e:
            print(f"Error sending email notification to {department}: {str(e)}")

# Detect and count negative and positive news in the first article
sentiment_1 = count_sentiments(news_article)
if sentiment_1 == 'Negative':
    negative_statements_count += 1
elif sentiment_1 == 'Positive':
    positive_statements_count += 1
department_1 = categorize_department(news_article)

# Check if department_1 is None and assign a default department
if department_1 is None:
    department_1 = 'Health Department'

# Check if any article has negative news and send notifications
if sentiment_1 == 'Negative':
    print(f"Sending SMS to {department_1}: {news_article}")
    send_sms_notification(news_article, department_1)
    send_email_notification(news_article, department_1)

# Detect and count negative and positive news in the second article
sentiment_2 = count_sentiments(news_article_2)
if sentiment_2 == 'Negative':
    negative_statements_count += 1
elif sentiment_2 == 'Positive':
    positive_statements_count += 1
department_2 = categorize_department(news_article_2)

# Check if department_2 is None and assign a default department
if department_2 is None:
    department_2 = 'Education Department'

# Check if any article has negative news and send notifications
if sentiment_2 == 'Negative':
    print(f"Sending SMS to {department_2}: {news_article_2}")
    send_sms_notification(news_article_2, department_2)
    send_email_notification(news_article_2, department_2)

# Detect and count negative and positive news in the third article
sentiment_3 = count_sentiments(news_article_3)
if sentiment_3 == 'Negative':
    negative_statements_count += 1
elif sentiment_3 == 'Positive':
    positive_statements_count += 1
department_3 = categorize_department(news_article_3)

# Check if department_3 is None and assign a default department
if department_3 is None:
    department_3 = 'Finance Department'

# Check if any article has negative news and send notifications
if sentiment_3 == 'Negative':
    print(f"Sending SMS to {department_3}: {news_article_3}")
    send_sms_notification(news_article_3, department_3)
    send_email_notification(news_article_3, department_3)

# Detect and count negative and positive news in the fourth article
sentiment_4 = count_sentiments(news_article_4)
if sentiment_4 == 'Negative':
    negative_statements_count += 1
elif sentiment_4 == 'Positive':
    positive_statements_count += 1
department_4 = categorize_department(news_article_4)

# Check if department_4 is None and assign a default department
if department_4 is None:
    department_4 = 'Transportation Department'

# Check if any article has negative news and send notifications
if sentiment_4 == 'Negative':
    print(f"Sending SMS to {department_4}: {news_article_4}")
    send_sms_notification(news_article_4, department_4)
    send_email_notification(news_article_4, department_4)

# Detect and count negative and positive news in the fifth article
sentiment_5 = count_sentiments(news_article_5)
if sentiment_5 == 'Negative':
    negative_statements_count += 1
elif sentiment_5 == 'Positive':
    positive_statements_count += 1
department_5 = categorize_department(news_article_5)

# Check if department_5 is None and assign a default department
if department_5 is None:
    department_5 = 'Agriculture Department'

# Check if any article has negative news and send notifications
if sentiment_5 == 'Negative':
    print(f"Sending SMS to {department_5}: {news_article_5}")
    send_sms_notification(news_article_5, department_5)
    send_email_notification(news_article_5, department_5)

# Print the counts for each article
print(f"Article 1: {negative_statements_count} negative statements, {positive_statements_count} positive statements")

# Print a message if no negative news is detected
if all(sentiment == 'Neutral' for sentiment in [sentiment_1, sentiment_2, sentiment_3, sentiment_4, sentiment_5]):
    print("No significant negative or positive statements detected in any article.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nandh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sending SMS to Health Department: 
The healthcare system is overwhelmed due to the surge in COVID-19 cases.
Hospitals are running out of beds and ventilators, putting patients' lives at risk.
Medical staff are exhausted and facing burnout from the prolonged crisis.
SMS Notification sent to Health Department. Message SID: SM06000bea44e7909e585f6e78d9f66892
Email notification sent to Health Department.
Sending SMS to Education Department: 
School closures continue due to the ongoing pandemic, causing disruptions in students' education.
Many students are struggling with remote learning and lack access to necessary resources.
Teachers are facing challenges in delivering quality education in virtual classrooms.
SMS Notification sent to Education Department. Message SID: SMf6af3b4023e800fa42a0432ae9f61eb0
Email notification sent to Education Department.
Sending SMS to Finance Department: 
The Indian Finance Department reported a significant decrease in tax revenues for the current fiscal yea